In [1]:
# import vars 
import integration.lf_exp as lib
import os
import numpy as np

# from kexp.util.data.load_atomdata import load_atomdata
# from kexp.analysis.plotting_1d import *

In [2]:
#create dictionary of each variable to include in the parameter space, its range of values and an initial value
var_dict = {
    'i_lf_tweezer_load_current': {
        'range': [12.0, 16.0], 
        'initial': 13.9},
    'v_pd_lf_tweezer_1064_ramp_end': {
        'range': [5.0, 9.3], 
        'initial': 9.2},
    'v_lf_tweezer_paint_amp_max': {
        'range': [-4.0, 3.0], 
        'initial': -2.7},
    'i_lf_tweezer_evap1_current': {
        'range': [12.0, 14.5], 
        'initial': 13.0},
    'v_pd_lf_tweezer_1064_rampdown_end': {
        'range': [0.35, 3.0], 
        'initial': 0.97},
    't_lf_tweezer_1064_rampdown': {
        'range': [0.02, 0.4], 
        'initial': 0.183},
    'i_lf_tweezer_evap2_current': {
        'range': [12.0, 14.], 
        'initial': 13.5},
    'v_pd_lf_tweezer_1064_rampdown2_end': {
        'range': [0.03, 0.13], 
        'initial': 0.11},
    't_lf_tweezer_1064_rampdown2': {
        'range': [0.1, 1.0], 
        'initial': 0.6}
}
print(len(var_dict))

9


In [3]:
# generate lists of parameter ranges and initial values to feed to MLOOP controller

init_params = [entry['initial'] for entry in var_dict.values()]
print(init_params)

ranges = [entry['range'] for entry in var_dict.values()]
print(ranges)

var_names = list(var_dict.keys())

[13.9, 9.2, -2.7, 13.0, 0.97, 0.183, 13.5, 0.11, 0.6]
[[12.0, 16.0], [5.0, 9.3], [-4.0, 3.0], [12.0, 14.5], [0.35, 3.0], [0.02, 0.4], [12.0, 14.0], [0.03, 0.13], [0.1, 1.0]]


In [4]:
#make table for vars
import pandas as pd


from kexp.config import expt_params

# Create an instance of the class
example_instance = expt_params.ExptParams()

# Prepare data for the table
data = []
for i, name in enumerate(var_names):
    default_value = getattr(example_instance, name, None)
    min_range, max_range = ranges[i]
    data.append([name, default_value, min_range, max_range])

# Create a DataFrame
df = pd.DataFrame(data, columns=["Variable Name", "Default Value", "Minimum Range Value", "Maximum Range Value"])

# Print the DataFrame
print(df)



                        Variable Name  Default Value  Minimum Range Value  \
0           i_lf_tweezer_load_current          13.75                12.00   
1       v_pd_lf_tweezer_1064_ramp_end           9.30                 5.00   
2          v_lf_tweezer_paint_amp_max          -1.80                -4.00   
3          i_lf_tweezer_evap1_current          13.50                12.00   
4   v_pd_lf_tweezer_1064_rampdown_end           1.67                 0.35   
5          t_lf_tweezer_1064_rampdown           0.18                 0.02   
6          i_lf_tweezer_evap2_current          13.00                12.00   
7  v_pd_lf_tweezer_1064_rampdown2_end           0.11                 0.03   
8         t_lf_tweezer_1064_rampdown2           0.25                 0.10   

   Maximum Range Value  
0                16.00  
1                 9.30  
2                 3.00  
3                14.50  
4                 3.00  
5                 0.40  
6                14.00  
7                 0.13  
8   

In [5]:
def main():
    #M-LOOP can be run with three commands
    
    #First create your interface
    interface = lib.CustomInterface(var_names)
    #Next create the controller. 
    #The controller must take the variables to be changed in the vars object
    #vars = Param()


    controller = lib.mlc.create_controller(interface, 
                                       controller_type = 'neural_net',
                                       max_num_runs = 2500,
                                       target_cost = -109090000000,
                                       num_params = len(var_dict),
                                       first_params = init_params, 
                                       min_boundary = np.transpose(ranges)[0],
                                       max_boundary = np.transpose(ranges)[1],
                                       interface_file_type = 'txt',               #file types of *exp_input.mat* and *exp_output.mat*
                                       controller_archive_file_type = 'txt',      #file type of the controller archive
                                       learner_archive_file_type = 'txt')      #file type of the learner archive
    controller.optimize()

main()

INFO     M-LOOP version 3.3.4
INFO     Optimization started.
INFO     Run: 0 (training)
INFO     params [13.9    9.2   -2.7   13.     0.97   0.183 13.5    0.11   0.6  ]
0 Run id: 42249
 1 values of dummy. 1 total shots. 3 total images expected.
andor
Waiting for camera ready.
42249
Acknowledged camera ready signal.
Camera is ready.

Sent: {'mask': 'spot', 'center': [1150, 918], 'phase': 0.0, 'dimension': 0}
-> mask: spot, dimension = 0 um, phase = 0.0 pi, x-center = 1150, y-center = 918

 Run ID: 42249
Parameters saved, data closed.
Done!
 
No ROI saved in run 42249.
ROI specified by Run ID. Attempting to load ROI...
ROI loaded from run 39249.
Using ROI loaded from run 39249.
-47.8302781068879
INFO     cost -47.8302781068879 +/- 10.0
INFO     Run: 1 (training)
INFO     params [13.80459299  5.93948716 -0.67185248 13.11102029  1.67018551  0.30830587
 12.29557953  0.10963806  0.52193013]
0 Run id: 42250
 1 values of dummy. 1 total shots. 3 total images expected.
andor
Acknowledged camera 

TypeError: object of type 'numpy.int32' has no len()

In [ ]:
def param_format(param_values,param_keys=var_names):
    lines = ""
    for i in range(len(param_keys)):
            lines += f"""
                self.p.{param_keys[i]} = {float(param_values[i])}"""
    print(lines)

param_format([-3.15221506,  0.10830731, -1.20206951, -2.93364217,  0.17977458,  0.1727242,
  1.96822209, -1.59931595, -3.91874333,  0.2,         0.09954919, 17.40341653])


                self.p.i_lf_tweezer_load_current = -3.15221506
                self.p.v_pd_lf_tweezer_1064_ramp_end = 0.10830731
                self.p.v_lf_tweezer_paint_amp_max = -1.20206951
                self.p.i_lf_tweezer_evap1_current = -2.93364217
                self.p.v_pd_lf_tweezer_1064_rampdown_end = 0.17977458
                self.p.t_lf_tweezer_1064_rampdown = 0.1727242
                self.p.i_lf_tweezer_evap2_current = 1.96822209
                self.p.v_pd_lf_tweezer_1064_rampdown2_end = -1.59931595
                self.p.t_lf_tweezer_1064_rampdown2 = -3.91874333


In [ ]:
def print_default_params(param_keys=var_names):
    from kexp.config import ExptParams
    p = ExptParams()
    lines = ""
    for key in param_keys:
            lines += f"""
                self.p.{key} = {float(vars(p)[key])}"""
    print(lines)
print_default_params()


                self.p.i_lf_tweezer_load_current = 13.75
                self.p.v_pd_lf_tweezer_1064_ramp_end = 9.3
                self.p.v_lf_tweezer_paint_amp_max = -1.8
                self.p.i_lf_tweezer_evap1_current = 13.5
                self.p.v_pd_lf_tweezer_1064_rampdown_end = 1.67
                self.p.t_lf_tweezer_1064_rampdown = 0.18
                self.p.i_lf_tweezer_evap2_current = 13.0
                self.p.v_pd_lf_tweezer_1064_rampdown2_end = 0.11
                self.p.t_lf_tweezer_1064_rampdown2 = 0.25


In [ ]:
def print_mloop_param_bounds(param_keys=var_names,param_bounds=var_space):
    lines = ""
    for i in range(len(param_keys)):
            lines += f"""
                self.p.{param_keys[i]} \u2208 {var_space[i]}"""
    print(lines)
print_mloop_param_bounds()

NameError: name 'var_space' is not defined

In [ ]:
def print_param_report(mloop_params,param_keys=var_names,param_bounds=var_space):
    from kexp.config import ExptParams
    p = ExptParams()
    lines = ""
    for i in range(len(param_keys)):
            line = []
            line.append(f"self.p.{param_keys[i]}".ljust(25,' '))
            line.append(f" \u2208 {var_space[i]}:".ljust(20,' '))
            line.append(f"mloop = {mloop_params[i]:1.3f},   ".ljust(18,' '))
            line.append(f"default = {vars(p)[param_keys[i]]:1.3f}".ljust(15,' '))
            s = ""
            for L in line:
                s += L
            lines += f"""
                {s}"""
    return lines
lines = print_param_report([-3.15221506,  0.10830731, -1.20206951, -2.93364217,  0.17977458,  0.1727242,
  1.96822209, -1.59931595, -3.91874333,  0.2,         0.09954919, 17.40341653])
print(lines)


                self.p.detune_push        ∈ [-5.4, 0.6]:     mloop = -3.152,   default = -2.400
                self.p.amp_push           ∈ [0.01, 0.2]:     mloop = 0.108,    default = 0.130
                self.p.detune_d2_c_2dmot  ∈ [-4.2, 1.8]:     mloop = -1.202,   default = -1.200
                self.p.detune_d2_r_2dmot  ∈ [-5.4, 0.6]:     mloop = -2.934,   default = -2.400
                self.p.amp_d2_c_2dmot     ∈ [0.01, 0.2]:     mloop = 0.180,    default = 0.188
                self.p.amp_d2_r_2dmot     ∈ [0.01, 0.2]:     mloop = 0.173,    default = 0.188
                self.p.v_2d_mot_current   ∈ [0.5, 5.0]:      mloop = 1.968,    default = 2.110
                self.p.detune_d2_c_mot    ∈ [-5.4, 0.6]:     mloop = -1.599,   default = -2.400
                self.p.detune_d2_r_mot    ∈ [-7.2, -1.2]:    mloop = -3.919,   default = -4.200
                self.p.amp_d2_c_mot       ∈ [0.01, 0.2]:     mloop = 0.200,    default = 0.188
                self.p.amp_d2_r_mot       ∈ 

In [ ]:
#### uncomment this and run it if u wanna delete the logs


# import os, shutil
# # Specify the path of the file to be deleted
# folder = r'C:\Users\bananas\code\k-exp\kexp\experiments\Mloop testing\M-LOOP_archives'
# file_path_2 = r'C:\Users\bananas\code\k-exp\kexp\experiments\Mloop testing\M-LOOP_logs'

# for filename in os.listdir(folder):
#     file_path = os.path.join(folder, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#     except Exception as e:
#         print('Failed to delete %s. Reason: %s' % (file_path, e))

# for filename in os.listdir(file_path_2):
#     file_path = os.path.join(folder, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#     except Exception as e:
#         print('Failed to delete %s. Reason: %s' % (file_path, e))